In [73]:
from math import nan
import pandas as pd
from charset_normalizer import from_path 
import re 
from pathlib import Path
from config import *
from config import log_config
from eda_analisys import *
from openpyxl import load_workbook 


Config load

In [74]:
# config
logger = log_config()
config.load_config() 


tables

In [75]:
logger.info("Início do processamento")

df_eda = pd.read_excel(config.eda_sheet_full_path,sheet_name='tables')
df_eda.columns = df_eda.columns.str.strip().str.lower()

# Abre a planilha para saída
try:
    wb = load_workbook(config.eda_sheet_full_path)
    tables_sheet = wb["tables"]
    fields_sheet = wb["fields"]
except FileNotFoundError:
    print("Erro: Arquivo 'eda.xlsx' não encontrado.")
    exit(1)

# Monta df_fields
data = list(fields_sheet.values)  # retorna tuplas, incluindo cabeçalho
headers = data[0]
rows = data[1:]
df_fields = pd.DataFrame(rows, columns=headers)

header_tables = headers = {str(cell.value).lower(): idx+1 for idx, cell in enumerate(tables_sheet[1]) if cell.value}

for row in tables_sheet.iter_rows(min_row=2):

    table_name = row[header_tables['table']-1].value
    if not pd.isna(table_name): 
        print(table_name)

        # file size
        data_file_full_path = Path(config.data_file_path / row[header_tables['file']-1].value) 
        row[header_tables['file_size']-1].value = file_size(table_name, data_file_full_path)

        # lines (qty) 
        row[header_tables['lines']-1].value = lines(table_name,data_file_full_path)

        # columns (qty) 
        row[header_tables['cols']-1].value = columns(table_name,data_file_full_path,config.separator)

        filter_for_table = df_fields['table'] == table_name
        df_table_fields = df_fields[filter_for_table]
        for index, field_row in df_table_fields.iterrows(): 
            excel_row = index + 2
            excel_col = df_fields.columns.get_loc('Tamanho') + 1 
            fields_sheet.cell(row=excel_row, column=excel_col).value = 'teste'
        

#for index, eda_row in df_eda.iterrows(): 

#for row in tables_sheet.iter_rows(values_only=True):




#     data_file_full_path = Path(config.data_file_path / eda_row['file']) 
#     excel_row = index + 2
    
#     # file size
#     excel_col = df_eda.columns.get_loc('file_size') + 1
#     tables_sheet.cell(row=excel_row, column=excel_col).value = file_size(eda_row['table'], data_file_full_path)

#     # lines (qty) 
#     excel_col = df_eda.columns.get_loc('lines') + 1
#     tables_sheet.cell(row=excel_row, column=excel_col).value = lines(eda_row['table'],data_file_full_path)

#     # columns (qty) 
#     excel_col = df_eda.columns.get_loc('cols') + 1
#     tables_sheet.cell(row=excel_row, column=excel_col).value = columns(eda_row['table'],data_file_full_path,config.separator) 

#     # columns existance
#     #eda_row['col_exists'] = column_existance(eda_row['table'],data_file_full_path,config.separator) 

#     # # columns duplicity
#     # eda_row['col_unique'] = column_duplicity(eda_row['table'],data_file_full_path,eda_config['separator']) 

#     # # pk unique
#     # eda_row['pk_integrity'] = pk_integrity(eda_row['table'],data_file_full_path,eda_config['separator']) 

    wb.save(config.eda_sheet_full_path)


ses_cias
ses_ramos
ses_seguros
